## install modules

In [29]:
import pandas as pd # allows us to work with tabular data
import requests # to send the API requests to Elsevier
import json # to read the JSON data that is returned by the APIs
import pickle # pickle files are a good way to save data for reuse in Python
from datetime import datetime
from langchain_community.chat_models import ChatOpenAI
from IPython.display import display, Image
from openai import OpenAI

In [18]:
pip install langchain-community

Note: you may need to restart the kernel to use updated packages.


## user  input 

In [20]:
api_key = ''
OPEN_API_KEY = ''

search_keyword = "5G"
start_year = 2015
end_year = 2023


In [21]:


search_string_parameters = {
    'ISSN' : '', # Limit results to results from a journal ISSN (add as a string). Leave = '' if no value.
    'AUTHOR-NAME' : '', # Search the author name field. lastname, firstname. e.g., Noble, Safiya
    'KEY' : '', # Add a keyword to search in the text of the article
    'PUBLISHER' : '', # Add a publisher name. e.g., Springer
    'EXACTSRCTITLE' : 'journal', # Add keywords that appear in the journal, book, or conference title. e.g., Informatics
    'AFFIL' : '', # add keywords that appear in the author's institutional affiliation. e.g., to find the University of Minnesota, search for University and Minnesota
    'TITLE' : search_keyword, # Add keywords that appear in the article or chapter title. Can use AND, OR, and AND NOT. e.g. cat AND dog 
    'OPENACCESS' : '1', # Add 1 to limit your results to open access articles or 0 for articles that are not open access
    'start_year' : start_year, # Limit your search to items published after this year. Leave = 0 if no value. YYYY , e.g., 1995
    'end_year' : end_year # Limit your search to items published before this year. Leave = 0 if no value. YYYY, e.g., 2020
}
def search_builder(search_string_parameters):
    ''' Concatenates a search string query formatted for the Scopus Search API.
    - search_string_parameters expects a python dictionary with keys aligned to API search fields.
    '''
    search_string = ''

    for k,v in search_string_parameters.items():
        if k == 'start_year' and v:
            if search_string != '':
                search_string += f' AND '
            search_string += f'PUBYEAR > {v}'
        elif k == 'end_year' and v:
            if search_string != '':
                search_string += f' AND '
            search_string += f'PUBYEAR < {v}'
        elif v:
            if search_string != '':
                search_string += f' AND '
            search_string += f'{k}({v})'

    return search_string

def create_url(search_string):
    '''Accepts a formatted search string that will be added to the Scopus Search API URL. 
    Requires a global api_key variable.
    Formats and returns a URL to send to the Scopus Search API.'''
    
    query = f'{search_string}'
    url_template = 'https://api.elsevier.com/content/search/scopus?query={query}&apiKey={api_key}'
    full_url = url_template.format(query=query, api_key=api_key)
    return full_url

def connect_to_endpoint(full_url, params={'cursor': '*'}, next_ = '*'):
    '''Accepts API URL with ISSN, default parameters, and next page cursor;
    Sends request to Scopus API and collects JSON results for each call;
    Returns r.json() for the ['search-results'] key.'''
    
    params['cursor'] = next_
    r = requests.get(full_url, params=params)
    r.raise_for_status()
    return r.json()['search-results'], r.headers

In [22]:
search_string = search_builder(search_string_parameters)
print(search_string)
full_url = create_url(search_string)
r_json, r_headers = connect_to_endpoint(full_url)

EXACTSRCTITLE(journal) AND TITLE(5G) AND OPENACCESS(1) AND PUBYEAR > 2015 AND PUBYEAR < 2023


In [23]:
print('Title:', r_json['entry'][0]['dc:title'], 
      '\nCreator:', r_json['entry'][0]['dc:creator'], 
      '\nPublication:', r_json['entry'][0]['prism:publicationName'],
      '\nDate:', r_json['entry'][0]['prism:coverDate'],
      '\nISSN:', r_json['entry'][0]['prism:url'], 
     '\nAbstract', r_json['entry'][0])

Title: Integrated Microwave and mm-Wave MIMO Antenna Module With 360° Pattern Diversity for 5G Internet of Things 
Creator: Hussain N. 
Publication: IEEE Internet of Things Journal 
Date: 2022-12-15 
ISSN: https://api.elsevier.com/content/abstract/scopus_id/85135752407 
DOI: 10.1109/JIOT.2022.3194676 
Abstract {'@_fa': 'true', 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85135752407'}, {'@_fa': 'true', '@ref': 'author-affiliation', '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85135752407?field=author,affiliation'}, {'@_fa': 'true', '@ref': 'scopus', '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85135752407&origin=inward'}, {'@_fa': 'true', '@ref': 'scopus-citedby', '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85135752407&origin=inward'}], 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85135752407', 'dc:identifier': 'SCOPUS_ID:851357524

In [24]:
dois = [entry.get('prism:doi') for entry in r_json.get('entry', [])]
print(dois[0])

10.1109/JIOT.2022.3194676


## 한건의 요청당 25개가 최대임 

In [26]:
print(len(dois))
dois = dois[:3]
print(len(dois))


3
3


## doi 추출 후 doi를 이용해 abstract를 가져와보자 

In [27]:
def create_url(doi):
    """Accepts a DOI as a string that will be added to the Abstract Retrieval API URL. 
    Requires a global api_key variable.
    Formats and returns a URL to send to the Abstract Retrieval API.
    """
    doi = f'{doi}'
    url_template = 'https://api.elsevier.com/content/abstract/doi/{doi}?&apiKey={api_key}'
    full_url = url_template.format(doi=doi, api_key=api_key)
    return full_url

def connect_to_endpoint(full_url):
    '''Input full_url from create_url function;
    Send request to Scopus Abstract Retrieval API
    Returns r.json response;'''
    
    r = requests.get(full_url, headers =  {'Accept': 'application/json'})
    r.raise_for_status()
    return r.json(), r.headers

In [30]:
chat_model = ChatOpenAI(openai_api_key=OPEN_API_KEY)
client = OpenAI(api_key=OPEN_API_KEY)

In [41]:
for index, doi in enumerate(dois):
    full_url = create_url(doi)
    r_json, r_headers = connect_to_endpoint(full_url)
    try:
        r_json = r_json['abstracts-retrieval-response']
        abstract = r_json['coredata']['dc:description']
        print(index)
        title = r_json['coredata']['dc:title']
        publication_name = r_json['coredata']['prism:publicationName']
        citedby_count = r_json['coredata']['citedby-count']          
        url = r_json['coredata']['prism:url']
        print('논문 제목  :  ' + title)
        print('출판지  :  ' + publication_name)
        print('인용횟수  :  ' + citedby_count)
        print(url)
#         result = chat_model.predict(abstract + " 이 내용을 100자 내외로 요약해줘")
#         print(result)
#         response = client.images.generate(
#                         model="dall-e-3",
#                         prompt=result,
#                         size="1024x1024",
#                         quality="standard",
#                         n=1,
#                     )

#         image_url = response.data[0].url
#         display(Image(url=image_url))
#         print(r_json['coredata']['dc:description'])
        
    except KeyError:
        ## abstract이 수록되지 않은 논문도 있음 
        print('예외가 발생한 부분')
        print(r_json['coredata']['dc:title'])
        print(index)

0
논문 제목  :  Integrated Microwave and mm-Wave MIMO Antenna Module With 360° Pattern Diversity for 5G Internet of Things
출판지  :  IEEE Internet of Things Journal
인용횟수  :  26
https://api.elsevier.com/content/abstract/scopus_id/85135752407
1
논문 제목  :  A Novel 4×4 MIMO Antenna Decoupled by T-Shaped Dummy Antenna with High Robustness for 5G Mobile Devices
출판지  :  Applied Computational Electromagnetics Society Journal
인용횟수  :  0
https://api.elsevier.com/content/abstract/scopus_id/85169569171
2
논문 제목  :  Wideband Decoupled 8-Element MIMO Mobile Phone Antenna for Sub-6GHz 5G NR Bands
출판지  :  Applied Computational Electromagnetics Society Journal
인용횟수  :  1
https://api.elsevier.com/content/abstract/scopus_id/85169565459


In [33]:
for index, doi in enumerate(dois):
    full_url = create_url(doi)
    r_json, r_headers = connect_to_endpoint(full_url)
    try:
        r_json = r_json['abstracts-retrieval-response']
        abstract = r_json['coredata']['dc:description']
        print(index)
        title = r_json['coredata']['dc:title']
        publication_name = r_json['coredata']['prism:publicationName]
        citedby_count = r_json['coredata']['citedby-count']                 
        print('논문 제목  :  ' + title)
        print('출판지  :  ' + publication_name)
        print('인용횟수  :  ' + citedby_count)
#         result = chat_model.predict(abstract + " 이 내용을 100자 내외로 요약해줘")
#         print(result)
#         response = client.images.generate(
#                         model="dall-e-3",
#                         prompt=result,
#                         size="1024x1024",
#                         quality="standard",
#                         n=1,
#                     )

#         image_url = response.data[0].url
#         display(Image(url=image_url))
#         print(r_json['coredata']['dc:description'])
        
    except KeyError:
        ## abstract이 수록되지 않은 논문도 있음 
        print('예외가 발생한 부분')
        print(r_json['coredata']['dc:title'])
        print(index)

0


C:\Users\user\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


5G IoT systems require antennas that cover a wide range of frequencies. We present a multiband antenna operating in microwave and mm-wave bands, with a 12-port MIMO configuration for 360° coverage. The antenna has good radiation efficiency and stable gain. It can be used in indoor scenarios for smart houses, offices, and vehicle-to-everything communications.


1
이 논문에서는 T자 모양의 더미 안테나를 사용하여 5G 모바일 기기용으로 설계된 안테나 쌍의 격리를 20db 이상으로 개선하고 안테나 위치의 변화에 강건함을 갖습니다. 제안된 안테나 쌍은 N77 대역의 3.8GHz에서 작동하며, 임피던스 대역폭은 280MHz입니다. T자 모양의 더미 안테나를 추가함으로써 제안된 안테나 쌍의 격리는 13db에서 30db로 향상되었으며, 안테나 위치가 변경되어도 높은 강건성을 갖습니다. T자 더미 안테나의 부하 요구 사항 계산 과정을 보여줍니다. 제안된 안테나 쌍을 복제하여 고 격리의 4×4 MIMO 안테나를 제안합니다. 측정 결과는 격리가 31db 이상인 MIMO 안테나를 보여주며, 평균 총 효율은 약 49%이고 -6db 패스밴드에서 Envelope Correlation Coefficient (ECC)는 0.05보다 낮습니다. 저자들의 최고 지식에 따르면, 대부분의 보고된 안테나 쌍과 비교하여 제안된 4×4 MIMO 안테나는 가장 높은 격리를 유지하면서도 컴팩트한 크기를 갖습니다.


2
이 연구에서는 서브-6G 모바일 단말기에 적합한 대역폭이 분리된 듀얼 안테나 쌍을 제안하고 설계했다. 이 다중입력 다중출력(MIMO) 안테나 쌍은 모바일 폰의 금속 프레임에 배열되며, 각 금속 프레임에 두 개의 안테나 쌍이 있다. 슬롯 구조 안테나 요소와 결함 접지 분리 구조를 사용하여 와이드 밴드와 높은 격리를 달성했다. 분리된 안테나 쌍을 기반으로 8개 요소 MIMO 안테나를 설계했으며, 시스템 표면에 지그재그 모양의 분리 결함 구조를 에칭함으로써 안테나 쌍 간의 결합을 효과적으로 감소시켰다. 안테나 어레이의 프로토타입을 성공적으로 제작하고 테스트했다. 시뮬레이션 및 실험 결과, 제안된 8개 요소 MIMO 안테나가 5G New Radio의 n77/n78/n79 밴드와 WLAN의 5 GHz 밴드를 완전히 커버할 수 있음을 보여준다. 안테나의 전체 효율은 50%에서 75%이며, 봉투 상관 계수(ECC)는 0.08 미만이고, 어떤 두 포트 간의 격리는 11 dB 이상이다.


In [ ]:
full_url = create_url(dois[0])

In [ ]:

r_json, r_headers = connect_to_endpoint(full_url)
r_json = r_json['abstracts-retrieval-response']
print(r_json['coredata']['dc:description'])